In [ ]:
import pandas as pd
import pyodbc as py
import numpy as np
import warnings
import argparse

import os
from datetime import date
import sys

# SQL BAGLANTI BOLUMU
sql_connection_str = (
    'DRIVER={SQL Server}; SERVER=85.111.25.185; DATABASE=veribase_ff_dw; UID=enes.guzelaydin; PWD=Enes.2023'
)

connection = py.connect(sql_connection_str)
cursor = connection.cursor()

SQL_Query = ("Select * from [dbo].[match_atc_data_full] ")
SQL_Query = pd.read_sql_query(SQL_Query, connection)

df = pd.read_sql_query("select * from [dbo].[match_atc_data_full]",connection)


In [ ]:
df.head()

In [ ]:

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def get_ratio(row):
    name = row['Prod']
    name1 = row['PROD_DESC']
    return fuzz.token_set_ratio(name, name1)

#df[df.apply(get_ratio, axis=1) > 70].head(10)

In [ ]:
df['score_prod'] =df.apply(get_ratio, axis=1)

In [ ]:
df.sort_values(by=['score_prod'],ascending=True).head(100)

In [ ]:
new=df.drop(df[df.score_prod < 50].index)
new.shape

In [ ]:
new.sort_values(by=['score_prod'],ascending=False).head(100)

In [21]:
import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import event

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": sql_connection_str})
engine = create_engine(connection_url)

In [22]:
@event.listens_for(engine, "before_cursor_execute")
def receive_before_cursor_execute(
        conn, cursor, statement, params, context, executemany
):
    if executemany:
        cursor.fast_executemany = True



In [23]:
df.to_sql('match_atc_data_full2'
                            ,engine
                            ,index=False
                            ,if_exists="append"
                            ,chunksize=10000
                            ,schema="dbo")


-28